In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load data
kc2 = pd.read_csv(r'/content/kc2_csv.csv')
print('kc2 shape: ', kc2.shape)
kc2.head()

kc2 shape:  (522, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,no
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,yes
2,415.0,59.0,50.0,51.0,1159.0,8411.31,0.01,103.53,81.24,870848.58,...,359,35,9,10,47.0,106.0,692.0,467.0,106.0,yes
3,230.0,33.0,10.0,16.0,575.0,3732.82,0.03,39.82,93.74,148644.06,...,174,15,34,5,23.0,67.0,343.0,232.0,65.0,yes
4,175.0,26.0,12.0,13.0,500.0,3123.96,0.03,29.48,105.96,92103.07,...,142,7,19,4,18.0,58.0,310.0,190.0,51.0,yes


In [3]:
kc2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loc               522 non-null    float64
 1   v(g)              522 non-null    float64
 2   ev(g)             522 non-null    float64
 3   iv(g)             522 non-null    float64
 4   n                 522 non-null    float64
 5   v                 522 non-null    float64
 6   l                 522 non-null    float64
 7   d                 522 non-null    float64
 8   i                 522 non-null    float64
 9   e                 522 non-null    float64
 10  b                 522 non-null    float64
 11  t                 522 non-null    float64
 12  lOCode            522 non-null    int64  
 13  lOComment         522 non-null    int64  
 14  lOBlank           522 non-null    int64  
 15  lOCodeAndComment  522 non-null    int64  
 16  uniq_Op           522 non-null    float64
 1

In [4]:
target = 'problems'
y = kc2[target]
x=kc2.drop(['problems'], axis=1)
x.shape

(522, 21)

In [5]:
pip install skfeature-chappers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 66 kB 5.0 MB/s 


In [6]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from skfeature.function.similarity_based import fisher_score

selector = SelectKBest(fisher_score.fisher_score, k=5)

x_reduced = selector.fit_transform(x,y)
x_reduced.shape



(522, 5)

In [7]:
cols = selector.get_support(indices=True)
selected_columns = x.iloc[:,cols].columns.tolist()
selected_columns

['n', 'e', 'b', 'lOCode', 'uniq_Opnd']

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.3, random_state=10)

In [9]:
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(522, 21)
(522,)
-------------------
(365, 5)
(365,)
(157, 5)
(157,)


In [10]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.8535031847133758
Th confusion matrix of SVM:
  [[125   2]
 [ 21   9]]


In [11]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[125   2]
 [ 21   9]]
              precision    recall  f1-score   support

          no       0.86      0.98      0.92       127
         yes       0.82      0.30      0.44        30

    accuracy                           0.85       157
   macro avg       0.84      0.64      0.68       157
weighted avg       0.85      0.85      0.82       157



In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.8152866242038217
The confusion matrix of KNN:
  [[124   3]
 [ 26   4]]
[[124   3]
 [ 26   4]]
              precision    recall  f1-score   support

          no       0.83      0.98      0.90       127
         yes       0.57      0.13      0.22        30

    accuracy                           0.82       157
   macro avg       0.70      0.55      0.56       157
weighted avg       0.78      0.82      0.77       157



In [13]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.7388535031847133
[[102  25]
 [ 16  14]]
              precision    recall  f1-score   support

          no       0.86      0.80      0.83       127
         yes       0.36      0.47      0.41        30

    accuracy                           0.74       157
   macro avg       0.61      0.63      0.62       157
weighted avg       0.77      0.74      0.75       157



In [14]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.8356164383561644

In [15]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

          no       0.85      0.95      0.90       127
         yes       0.60      0.30      0.40        30

    accuracy                           0.83       157
   macro avg       0.73      0.63      0.65       157
weighted avg       0.80      0.83      0.80       157

[[121   6]
 [ 21   9]]
